In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
import xarray as xr

In [3]:
def time_mean(ds):
    coords = ds.coords
    weights = xr.DataArray(
        ds.time_bounds.diff("time_bounds").values.astype("float64"),
        coords={"time":ds.time},
        dims=("time",)
    )
    ds_mean = ds.weighted(weights).mean("time")
    ds_mean = ds_mean.assign_coords({c:coord for (c,coord) in coords.items() if c not in ds_mean.coords})
    return ds_mean

In [8]:
toSv = 1e-6/1035.
for model in ["CM4Xp25", "CM4Xp125"]:
    wmt = xr.open_zarr(f"../../data/wmb_{model}_global_2340-2349.zarr") * toSv
    wmt_mean = time_mean(wmt.sel(exp="control"))
    wmt_mean_subset = wmt_mean[[
        'mass_tendency',
        'mass_source',
        'boundary_fluxes',
        'diffusion',
        'spurious_numerical_mixing',
        'layer_mass',
        
    ]].assign_coords(wmt_mean.coords).drop_dims(["xh", "yh"])
    wmt_mean_subset.load()
    wmt_mean_subset.to_netcdf(f"../../data/plotting_ready/wmb_{model}_global_2340-2349_time-mean.nc", mode="w")

In [9]:
wmt_mean_subset

<xarray.Dataset> Size: 34kB
Dimensions:                    (sigma2_l_target: 500, time_bounds: 121,
                                sigma2_i_target: 501, time: 120)
Coordinates:
    exp                        <U7 28B 'control'
  * sigma2_l_target            (sigma2_l_target) float64 4kB 0.0 0.1 ... 49.9
  * time_bounds                (time_bounds) object 968B 0591-01-01 00:00:00 ...
  * sigma2_i_target            (sigma2_i_target) float64 4kB -0.05 ... 49.95
  * time                       (time) object 960B 0591-01-16 12:00:00 ... 060...
Data variables:
    mass_tendency              (sigma2_l_target) float64 4kB -0.008704 ... -1...
    mass_source                (sigma2_l_target) float64 4kB -0.008283 ... -7...
    boundary_fluxes            (sigma2_l_target) float64 4kB 0.0 ... -0.0005843
    diffusion                  (sigma2_l_target) float64 4kB 0.0 ... -8.618e-05
    spurious_numerical_mixing  (sigma2_l_target) float64 4kB -0.0004212 ... 0...
    layer_mass                 (sigma2_l_target) float64 4kB 0.0 0.0 ... 626.4